# Deploy GLM 4 chat model to Sagemaker 

(Under development, dont try.)

Deploy a model use OpenAI API schema to interact with.

Model url: https://huggingface.co/THUDM/glm-4-9b-chat

## Install and upgrade dependencies

In [ ]:
# init sagemaker parameters
import boto3
import sagemaker
from sagemaker import Model, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_code_prefix = "chatglm4/accelerate"


The inference codes is placed in the ./code folder.

In [ ]:
!rm -f model.tar.gz
!tar czvf model.tar.gz -C code .

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar uploaded to --- > {s3_code_artifact}")

In [ ]:
# retrieve SageMaker LMI container image URI
image_uri = sagemaker.image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.28.0"
)


print(image_uri)

model = Model(image_uri=image_uri, model_data=s3_code_artifact, role=role)

instance_type = "ml.g5.2xlarge"  # "ml.g5.2xlarge" - #single GPU. really need one GPU for this since tensor split is '1'

endpoint_name = "glm-4-chat"

print("模型部署过程大约需要 7~8 分钟，请等待" + "."*20)

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=900,
)

print("模型部署已完成，可以继续执行后续步骤" + "."*20)